<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary 代码 for 这个 <一个 href="http://mng.bz/orYv">构建 一个 大语言模型 From Scratch</一个> book by <一个 href="https://sebastianraschka.com">Sebastian Raschka</一个><br>
<br>代码 repository: <一个 href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</一个>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<一个 href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></一个>
</td>
</tr>
</table>

# Generating 一个 Preference 数据集 With Llama 3.1 70B 和 Ollama

- Preference finetuning is 一个 处理 to align 一个 instruction-finetuned 大语言模型 with human preferences
- 那里 are multiple ways to 创建 一个 数据集 for preference finetuning 一个 大语言模型
  1. 我们 使用 这个 instruction-finetuned 大语言模型 to 生成 multiple responses 和 have humans rank them based on their preference 和/或者 given preference criteria
  2. 我们 使用 这个 instruction-finetuned 大语言模型 to 生成 multiple responses 和 have LLMs rank them based on given preference criteria
  3. 我们 使用 一个 大语言模型 to 生成 preferred 和 dispreferred responses given certain preference criteria
- In 这个 笔记本, 我们 consider approach 3
- 这个 笔记本 uses 一个 70-billion-参数 Llama 3.1-Instruct 模型 through ollama to 生成 preference labels for 一个 instruction 数据集
- 这个 expected format of 这个 instruction 数据集 is as follows:


### 输入

```json
[
    {
        "instruction": "什么 is 这个 state capital of California?",
        "输入": "",
        "输出": "这个 state capital of California is Sacramento.",
    },
    {
        "instruction": "Provide 一个 synonym for 'fast'.",
        "输入": "",
        "输出": "一个 synonym for 'fast' is 'quick'.",
    },
    {
        "instruction": "什么 is 这个 capital of Greece?",
        "输入": "",
        "输出": "这个 capital of Greece is Athens.",

    },
...
]
```

这个 输出 数据集 will look as follows, 哪里 more polite responses are preferred (`'chosen'`), 和 more impolite responses are dispreferred (`'rejected'`):

```json
[
    {
        "instruction": "什么 is 这个 state capital of California?",
        "输入": "",
        "输出": "这个 state capital of California is Sacramento.",
        "rejected": "Look, 这个 state capital of California is obviously Sacramento.",
        "chosen": "这个 state capital of California is Sacramento."
    },
    {
        "instruction": "Provide 一个 synonym for 'fast'.",
        "输入": "",
        "输出": "一个 synonym for 'fast' is 'quick'.",
        "chosen": "一个 suitable alternative to 'fast' would be 'quick'.",
        "rejected": "一个 synonym for 'fast' is 'quick'."
    },
    {
        "instruction": "什么 is 这个 capital of Greece?",
        "输入": "",
        "输出": "这个 capital of Greece is Athens.",
        "chosen": "I'd be happy to help! 这个 capital of Greece is indeed Athens.",
        "rejected": "这个 capital of Greece is Athens."
    },
...
]
```

### 输出




- 这个 代码 doesn't require 一个 GPU 和 runs on 一个 laptop given enough RAM

In [1]:
from importlib.metadata import version

pkgs = ["tqdm",    # Progress bar
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.66.4


## Installing Ollama 和 Downloading Llama 3.1

- Ollama is 一个 application to 运行 LLMs efficiently
- 它 is 一个 wrapper around [llama.cpp](https://github.com/ggerganov/llama.cpp), 哪个 implements LLMs in pure C/C++ to maximize efficiency
- Note 那个 它 is 一个 tool for using LLMs to 生成 text (推理), not 训练 或者 finetuning LLMs
- Prior to running 这个 代码 below, 安装 ollama by visiting [https://ollama.com](https://ollama.com) 和 following 这个 instructions (for instance, clicking on 这个 "Download" button 和 downloading 这个 ollama application for your operating system)

- For macOS 和 Windows users, click on 这个 ollama application 你 downloaded; 如果 它 prompts 你 to 安装 这个 command line usage, say "yes"
- Linux users can 使用 这个 安装 command provided on 这个 ollama website

- In general, before 我们 can 使用 ollama from 这个 command line, 我们 have to either 开始 这个 ollama application 或者 运行 `ollama serve` in 一个 separate terminal

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/ollama-eval/ollama-serve.webp?1">


- With 这个 ollama application 或者 `ollama serve` running, in 一个 different terminal, on 这个 command line, 执行 这个 following command to try out 这个 70-billion-参数 Llama 3.1 模型 

```bash
# 70B 模型
ollama 运行 llama3.1:70b
```


这个 输出 looks like as follows:

```
$ ollama 运行 llama3.1:70b
pulling manifest
pulling aa81b541aae6... 100% ▕████████████████▏ 39 GB
pulling 8cf247399e57... 100% ▕████████████████▏ 1.7 KB
pulling f1cd752815fc... 100% ▕████████████████▏ 12 KB
pulling 56bb8bd477a5... 100% ▕████████████████▏ 96 B
pulling 3c1c2d3df5b3... 100% ▕████████████████▏ 486 B
verifying sha256 digest
writing manifest
removing any unused layers
success
```

- Note 那个 `llama3.1:70b` refers to 这个 instruction finetuned 70-billion-参数 Llama 3.1 模型

- Alternatively, 你 can also 使用 这个 smaller, more resource-effiicent 8-billion-parameters Llama 3.1 模型, by replacing `llama3.1:70b` with `llama3.1`

- After 这个 download has been completed, 你 will see 一个 command line prompt 那个 allows 你 to chat with 这个 模型

- Try 一个 prompt like "什么 do llamas eat?", 哪个 should 返回 一个 输出 similar to 这个 following:

```
>>> 什么 do llamas eat?
Llamas are ruminant animals, 哪个 means they have 一个 four-chambered 
stomach 和 eat plants 那个 are high in fiber. In 这个 wild, llamas 
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall 
grasses, wheat, oats, 和 barley.
```

- 你 can 结束 这个 session using 这个 输入 `/bye`

## Using Ollama's REST API

- 现在, 一个 alternative way to interact with 这个 模型 is via its REST API in Python via 这个 following 函数
- Before 你 运行 这个 接下来 cells in 这个 笔记本, make sure 那个 ollama is still running, as described above, via
  - `ollama serve` in 一个 terminal
  - 这个 ollama application
- 接下来, 运行 这个 following 代码 cell to query 这个 模型

- 首先, 让我们 try 这个 API with 一个 simple 示例 to make sure 它 works as intended:

In [2]:
import urllib.request
import json


def query_model(prompt, model="llama3.1:70b", url="http://localhost:11434/api/chat"):
    # 创建 这个 data payload as 一个 dictionary
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0,
        }
    }

    # 转换 这个 dictionary to 一个 JSON formatted string 和 encode 它 to bytes
    payload = json.dumps(data).encode("utf-8")

    # 创建 一个 request object, setting 这个 方法 to POST 和 adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send 这个 request 和 capture 这个 response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read 和 decode 这个 response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data


result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet consists of:

1. **Grasses**: Various types of grasses, including timothy grass, orchard grass, and brome grass.
2. **Hay**: High-quality hay, such as alfalfa or clover hay, is a staple in a llama's diet.
3. **Leaves**: Leaves from trees and shrubs, like willow, cottonwood, and mesquite, are also eaten.
4. **Fruits and vegetables**: Llamas enjoy fruits like apples, carrots, and sweet potatoes, as well as leafy greens like kale and spinach.
5. **Grains**: In moderation, llamas can eat grains like oats, barley, and corn.

It's essential to note that llamas have a unique digestive system, with a three-part stomach and a large cecum (a specialized part of the large intestine). This allows them to break down and extract nutrients from plant material more efficiently than many other animals.

A typical llama diet might consist of:

* 1-2% of their body weight in hay per day
* 0.5-1% of their body w

## 加载 JSON Entries

- 现在, 让我们 获取 to 这个 data generation part
- 这里, for 一个 hands-on 示例, 我们 使用 这个 `instruction-data.json` file 那个 我们 originally used to instruction-finetune 这个 模型 in 第 7:

In [3]:
from pathlib import Path

json_file = Path("..", "01_main-chapter-code", "instruction-data.json")

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 1100


- 这个 structure of 这个 file is as follows, 哪里 我们 have 这个 given response in 这个 测试 数据集 (`'输出'`) 那个 我们 trained 这个 模型 to 生成 via instruction finetuning based on 这个 `'输入'` 和 `'instruction'`

In [4]:
json_data[0]

{'instruction': 'Evaluate the following phrase by transforming it into the spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".'}

- Below is 一个 small utility 函数 那个 formats 这个 instruction 和 输入:

In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### 输入:\n{entry['输入']}" 如果 entry["输入"] else ""
    instruction_text + input_text

    return instruction_text + input_text

- 现在, 让我们 try 这个 ollama API to 生成 一个 `'chosen'` 和 `'rejected'` response for preference tuning 一个 模型
- 这里, to for illustration purposes, 我们 创建 answers 那个 are more 或者 less polite


In [6]:
import random


for entry in json_data[:5]:
    
    politeness = random.choice(["polite", "impolite"])    
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"slightly rewrite the output to be more {politeness}."
        "Keep the modification minimal."
        "Only return return the generated response and nothing else."
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print(f"\n{politeness} response:")
    print(">>", query_model(prompt))    


Dataset response:
>> The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".

impolite response:
>> The spelling of the given phrase "freind" is flat out wrong, get it together, the correct spelling is "friend".

Dataset response:
>> He goes to the park every day.

polite response:
>> He goes to the park daily, if I'm not mistaken.

Dataset response:
>> 45 kilometers is 45000 meters.

polite response:
>> 45 kilometers is equivalent to 45000 meters.

Dataset response:
>> Although it was raining, they went for a walk.

polite response:
>> Although it was raining outside, they still decided to go for a walk.

Dataset response:
>> 1, 4, 9, 16, 25, 36, 49, 64, 81, 100.

impolite response:
>> Here are your precious square numbers: 1, 4, 9, 16, 25, 36, 49, 64, 81, 100.


- 如果 我们 find 那个 这个 generated responses above look reasonable, 我们 can go to 这个 接下来 step 和 应用 这个 prompt to 这个 whole 数据集
- 这里, 我们 添加 一个 `'chosen'` key for 这个 preferred response 和 一个 `'rejected'` response for 这个 dispreferred response

In [7]:
import random
from tqdm import tqdm

def generate_model_responses(json_data):

    for i, entry in enumerate(tqdm(json_data, desc="Writing entries")):
        politeness = random.choice(["polite", "impolite"])    
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"slightly rewrite the output to be more {politeness}."
            "Keep the modification minimal."
            "Only return return the generated response and nothing else."
        )
        response = query_model(prompt)
        
        if politeness == "polite":
            json_data[i]["chosen"] = response
            json_data[i]["rejected"] = entry["output"]
        else:
            json_data[i]["rejected"] = response
            json_data[i]["chosen"] = entry["output"]    

- 让我们 现在 应用 这个 evaluation to 这个 whole 数据集 和 计算 这个 average score of each 模型 (这个 takes about 1 minute per 模型 on 一个 M3 MacBook Air laptop)
- Note 那个 ollama is not fully deterministic across operating systems (as of 这个 writing) so 这个 numbers 你 are getting might slightly differ from 这个 ones shown below

In [8]:
generate_model_responses(json_data)

Writing entries: 100%|██████████| 1100/1100 [17:20<00:00,  1.06it/s]


In [10]:
with open("instruction-data-with-preference.json", "w") as file:
    json.dump(json_data, file, indent=4)